In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_identity.csv')
train_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_transaction.csv')
test_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_identity.csv')
test_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_transaction.csv')
test_identity.columns = [col.replace('-','_')for col in test_identity.columns]

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [6]:
del train_identity,test_identity,train_transaction,test_transaction 

# 전처리 [컬럼drop(null_80%이상)]

In [7]:
dff_column_to_drop = train.columns[(train.isna().sum() / len(train) > 0.2)]

In [8]:
dff_column_to_drop=dff_column_to_drop.drop(['id_30','id_31','id_32','id_33','id_34','DeviceType','DeviceInfo'])

In [9]:
def preprocess(df) :
    df = df.drop(columns=dff_column_to_drop).copy()
    # object 범주형 자료 결측치 - 최빈값으로 채워 넣기
    # df.ProductCD.value_counts().index # ['W', 'C', 'R', 'H', 'S']
    df.ProductCD.fillna(df.ProductCD.mode(), inplace=True)
    from sklearn.preprocessing import LabelEncoder
    lbl_enc = LabelEncoder()
    lbl_enc.fit(df[['ProductCD']])
    df['ProductCD'] = lbl_enc.transform(df[['ProductCD']])
    # df.card4.value_counts().index # ['visa', 'mastercard', 'american express', 'discover']
    df.card4.fillna(df.card4.mode(), inplace=True)
    lbl_enc.fit(df[['card4']])
    df['card4'] = lbl_enc.transform(df[['card4']])
    # df.card6.value_counts().index # ['debit', 'credit', 'debit or credit', 'charge card']
    df.card6.fillna(df.card6.mode(), inplace=True)
    lbl_enc.fit(df[['card6']])
    df['card6'] = lbl_enc.transform(df[['card6']])
    # df.P_emaildomain.value_counts().index    # [ gmail yahoo etc   ]
    df.P_emaildomain.fillna(df.P_emaildomain.mode(), inplace=True)
    lbl_enc.fit(df[['P_emaildomain']])
    df['P_emaildomain'] = lbl_enc.transform(df[['P_emaildomain']])
    # id_30
    df.id_30.fillna(df.id_30.mode(), inplace=True)
    lbl_enc.fit(df[['id_30']])
    df['id_30'] = lbl_enc.transform(df[['id_30']])
    #id_31
    df.id_31.fillna(df.id_31.mode(), inplace=True)
    lbl_enc.fit(df[['id_31']])
    df['id_31'] = lbl_enc.transform(df[['id_31']])
    #id_33
    df.id_33.fillna(df.id_33.mode(), inplace=True)
    lbl_enc.fit(df[['id_33']])
    df['id_33'] = lbl_enc.transform(df[['id_33']])
    #id_34
    df.id_34.fillna(df.id_34.mode(), inplace=True)
    lbl_enc.fit(df[['id_34']])
    df['id_34'] = lbl_enc.transform(df[['id_34']])
    #DeviceType
    df.DeviceType.fillna(df.DeviceType.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceType']])
    df['DeviceType'] = lbl_enc.transform(df[['DeviceType']])
    #DeviceInfo
    df.DeviceInfo.fillna(df.DeviceInfo.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceInfo']])
    df['DeviceInfo'] = lbl_enc.transform(df[['DeviceInfo']])
    

    #NULL값이 있는지 한눈에 보기
    display(np.sum(df.isnull() ).to_frame().transpose() )
    display(f'지금 {np.sum(np.sum(df.isnull()))} 개의 null값이 있습니다.' )


    # continuous 연속형 자료 결측치 추가
    # 방법 1 - 중앙값으로 채워 넣기
    temp_list_1 = df.columns[df.isnull().sum() > 0]  # 결측치가 있는 컬럼을 골라서 리스트에 넣기
    for aa in temp_list_1 :
        df[aa].fillna(df[aa].median(), inplace=True)  # 중앙값으로 채워 넣기
    # df.info(verbose=True, show_counts=True)

    print("Null :", df.isnull().sum().sum() )

    return df

In [10]:
train = preprocess(train)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,0,8933,1565,0,4259,...,12,12,12,0,0,512954,0,0,0,0


'지금 6082443 개의 null값이 있습니다.'

Null : 0


In [11]:
test = preprocess(test)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,8654,3002,0,4547,0,...,3,3,3,0,0,436020,0,0,0,0


'지금 1500255 개의 null값이 있습니다.'

Null : 0


In [12]:
y = train['isFraud']
x = train.drop('isFraud',axis=1)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=0xC0FFEE)  # 42, 0xC0FFEE

In [14]:
#결측치만 채운 데이터 성능 확인
from sklearn.metrics import accuracy_score, precision_score,recall_score,confusion_matrix

def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision  = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy,precision,recall))

In [ ]:
def get_model_train_eval(model,ftr_train=None,ftr_test=None,tgt_train=None,tgt_test=None):
    model.fit(ftr_train,tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:,1]
    get_clf_eval(tgt_test,pred)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=x_train,ftr_test=x_val,tgt_train=y_train,tgt_test=y_val)

# feature engineering

In [15]:
#transactionAmt
# 클립하여 0부터 5000사이의 값으로 봅니다
train['TransactionAmt'] = train['TransactionAmt'].clip(0,5000)
test['TransactionAmt']  = test['TransactionAmt'].clip(0,5000)

# 거래 액수가 일반적인지 아닌지를 봅니다
train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)

print('train')
print(train.TransactionAmt_check.value_counts())
print(' ')
print('test')
print(test.TransactionAmt_check.value_counts())

train
1    527684
0     62856
Name: TransactionAmt_check, dtype: int64
 
test
1    490910
0     15781
Name: TransactionAmt_check, dtype: int64


In [16]:
# v's pca 
from sklearn.preprocessing import StandardScaler

v_train = train.loc[:,'V12':'V321']
v_train = v_train.fillna(-999)
v_train = StandardScaler().fit_transform(v_train)
v_test = test.loc[:,'V12':'V321']
v_test = v_test.fillna(-999)
v_test = StandardScaler().fit_transform(v_test)


columns = ['PCA_V1','PCA_V2','PCA_V3','PCA_V4','PCA_V5','PCA_V6','PCA_V7','PCA_V8','PCA_V9','PCA_V10','PCA_V11','PCA_V12','PCA_V13','PCA_V14','PCA_V15','PCA_V16','PCA_V17','PCA_V18','PCA_V19','PCA_V20','PCA_V21','PCA_V22','PCA_V23','PCA_V24','PCA_V25','PCA_V26','PCA_V27','PCA_V28','PCA_V29','PCA_V30']
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
principalComponents_train = pca.fit_transform(v_train)
principalDF_train = pd.DataFrame(data=principalComponents_train,columns=columns)
principalComponents_test = pca.fit_transform(v_test)
principalDF_test = pd.DataFrame(data=principalComponents_test,columns=columns)

train = train.drop(train.loc[:,'V12':'V321'].columns,axis=1)
test = test.drop(test.loc[:,'V12':'V321'].columns,axis=1)

train = pd.concat([train,principalDF_train],axis=1)
test = pd.concat([test,principalDF_test],axis=1)

# over sampling

In [17]:
#feature engineering 한 데이터 셋에 over sampling 해주기
y = train['isFraud']
x = train.drop('isFraud',axis=1)

In [18]:
from imblearn.over_sampling import SMOTE

def smote(x,y):
    smote = SMOTE(random_state=0)
    x_over ,y_over = smote.fit_resample(x,y)
    print('smote 적용 후 레이블 값 분포:\n',pd.Series(y_over).value_counts())
    return x_over, y_over

In [19]:
x_over ,y_over = smote(x,y)

smote 적용 후 레이블 값 분포:
 0    569877
1    569877
Name: isFraud, dtype: int64


# column 조합으로 주요 식별 정보 만들기

In [20]:
import datetime
def date_columns(all_data):
    START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
    all_data['DT_time'] = all_data['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x))).astype('object')
    all_data['diff_days'] = all_data['D1'].fillna(0).apply(lambda x: (datetime.timedelta(days = x))).astype('object')
    all_data['client_startdate_date'] = (all_data['DT_time']- all_data['diff_days']).astype('object')
    all_data['client_startdate'] = (all_data['DT_time']- all_data['diff_days']).apply(lambda x:str(x.date())).astype('object')

In [21]:
date_columns(x_over)
date_columns(test)

In [22]:
import hashlib
def card_info_hash(x):
    s = (str(int(x['card1']))+
         str(int(x['card2']))+
         str(int(x['card3']))+
         str(x['card4'])+
         str(int(x['card5']))+
         str(x['card6']))
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [23]:
def device_hash(x):
    s =  str(x['id_30'])+str(x['id_31'])+str(x['id_32'])+str(x['id_33'])+str( x['DeviceType'])+ str(x['DeviceInfo'])
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()[0:15]
    return h

In [24]:
x_over['card_hash'] = x_over.apply(lambda x: card_info_hash(x), axis=1)

In [25]:
test['card_hash'] = test.apply(lambda x: card_info_hash(x), axis=1)

In [26]:
x_over['device_hash'] = x_over.apply(lambda x: device_hash(x), axis=1)

In [27]:
test['device_hash'] = test.apply(lambda x: device_hash(x), axis=1)

In [28]:
#label encoding
from sklearn import preprocessing
def label_encoding(df):
    for f in df.columns:
        if df[f].dtype == 'object':
            le = preprocessing.LabelEncoder()
            le.fit(list(df[f].values))
            df[f] = le.transform(list(df[f].values))
        else :
            pass

In [29]:
label_encoding(x_over)
label_encoding(test)

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_over,y_over, test_size=0.2, random_state=0xC0FFEE)  # 42, 0xC0FFEE

In [ ]:
#over sampling 한 데이터 셋 성능 보기
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=x_train,ftr_test=x_val,tgt_train=y_train,tgt_test=y_val)

# feature selection

변수간 상관도에 따른 컬럼 제거

- Pearson R 상관 계수를 구한다
- 상관계수가 0.9 이상인 컬럼들 중 가장 큰 컬럼을 제거해 본다
- 컬럼들 간 조합 생성

In [31]:
from itertools import combinations
from scipy import stats
comb_feat = np.array(list(combinations(x_over,2)))
corr_feat = np.array([])
for comb in comb_feat:
    corr = stats.pearsonr(x_over[comb[0]],x_over[comb[1]])[0]
    corr_feat = np.append(corr_feat,corr)

In [32]:
high_corr_num =comb_feat[np.abs(corr_feat)>= 0.9]
high_corr_num

array([['TransactionID', 'TransactionDT'],
       ['TransactionID', 'DT_time'],
       ['TransactionDT', 'DT_time'],
       ['C1', 'C2'],
       ['C1', 'C4'],
       ['C1', 'C6'],
       ['C1', 'C7'],
       ['C1', 'C8'],
       ['C1', 'C10'],
       ['C1', 'C11'],
       ['C1', 'C12'],
       ['C1', 'C14'],
       ['C2', 'C4'],
       ['C2', 'C6'],
       ['C2', 'C7'],
       ['C2', 'C8'],
       ['C2', 'C10'],
       ['C2', 'C11'],
       ['C2', 'C12'],
       ['C2', 'C14'],
       ['C4', 'C6'],
       ['C4', 'C7'],
       ['C4', 'C8'],
       ['C4', 'C10'],
       ['C4', 'C11'],
       ['C4', 'C12'],
       ['C4', 'C14'],
       ['C5', 'C9'],
       ['C6', 'C7'],
       ['C6', 'C8'],
       ['C6', 'C10'],
       ['C6', 'C11'],
       ['C6', 'C12'],
       ['C6', 'C14'],
       ['C7', 'C8'],
       ['C7', 'C10'],
       ['C7', 'C11'],
       ['C7', 'C12'],
       ['C8', 'C10'],
       ['C8', 'C11'],
       ['C8', 'C12'],
       ['C8', 'C14'],
       ['C10', 'C11'],
       ['C10', 'C1

# 필요 없는 columns 제거

In [33]:
drop_columns = ['C1','C2','C4','C6','C7','C8','C11','TransactionID','card1', 'card2', 'card3', 'card4', 'card5', 'card6','id_30', 'id_31',
       'id_32', 'id_33', 'id_34', 'DeviceType', 'DeviceInfo','DT_time', 'diff_days', 'client_startdate_date']

In [34]:
x_over = x_over.drop(drop_columns,axis=1)

In [35]:
test = test.drop(drop_columns,axis=1)

# HyperOpt를 이용한 하이퍼 파라미터 최적화

2.lightgbm

In [ ]:
from lightgbm import LGBMClassifier

from hyperopt import hp
lgbm_search_space={
    'num_leaves':hp.quniform('num_leaves',32,64,1),
    'max_depth':hp.quniform('max_depth',100,160,1),
    'min_child_samples':hp.quniform('min_child_samples',60,100,1),
    'subsample':hp.uniform('subsample',0.7,1),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2)
    
}

In [ ]:
from sklearn.model_selection import cross_val_score
def objective_func(search_space):
    lgbm_clf = LGBMClassifier(n_estimators=100,
                             num_leaves=int(search_space['num_leaves']),
                             max_depth = int(search_space['min_child_samples']),
                             subsample = search_space['subsample'],
                             learning_rate=search_space['learning_rate'])
    accuracy = cross_val_score(lgbm_clf,x_over,y_over,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}

In [ ]:
from hyperopt import fmin,tpe,Trials
from hyperopt import STATUS_OK

trials=Trials()

best = fmin(fn=objective_func,space=lgbm_search_space,algo=tpe.suggest,
           max_evals=50,trials=trials, rstate=np.random.default_rng(seed=30))
print('best:',best)

3.catboost

In [ ]:
cat_search_space={
        'depth': hp.quniform("depth", 1, 6, 1),
        'border_count': hp.uniform ('border_count', 32, 255),
        'learning_rate': hp.loguniform('learning_rate', -5.0, -2),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 8),
}

In [ ]:
from catboost import CatBoostClassifier
def objective_func(search_space):
    cat_clf = CatBoostClassifier(n_estimators=100,
                                learning_rate=search_space['learning_rate'],
                                depth=int(search_space['depth']),
                                eval_metric='AUC',
                                l2_leaf_reg=search_space['l2_leaf_reg'],
                                early_stopping_rounds=50,
                                border_count=int(search_space['border_count']),
                                verbose=False
                                        )
    accuracy = cross_val_score(cat_clf,x_over,y_over,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}
    

In [ ]:
trials = Trials()
best_2 = fmin(
        fn=objective_func,
        space=cat_search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials,
        )
print('best:',best_2)

# CV 세트 기반의 스태킹

In [37]:
#lgbm 최적 하이퍼 파라미터 이용
'''best:{'learning_rate': 0.012882961856246808, 'max_depth': 101.0, 
'min_child_samples': 100.0, 'num_leaves': 53.0, 'subsample': 0.9440498280474866}'''
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(
    n_estimators = 400,
    learning_rate = 0.012882961856246808,
    max_dept= 101,
    min_child_samples= 100,
    num_leaves = 53,
    subsample = 0.9440498280474866,
)

In [38]:
#catboost 최적 하이퍼 파라미터 이용
'''{'border_count': 32.9664754169335, 'depth': 1.0, 
'l2_leaf_reg': 3.631584119344952, 'learning_rate': 0.008568746451603016}'''
from catboost import CatBoostClassifier
cat_wrapper = CatBoostClassifier(
    n_estimators = 400,
    border_count = 32,
    depth = 1,
    l2_leaf_reg = 3.631584119344952,
    learning_rate = 0.008568746451603016
)

In [39]:
x_over = np.array(x_over)
y_over = np.array(y_over)
test = np.array(test)
#x_over이 is_Fraud가 이미 없어서 지우고 돌림

In [40]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict_proba(X_te)[:,1].reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict_proba(X_test_n)[:,1]
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [41]:
cat_train, cat_test = get_stacking_base_datasets(cat_wrapper, x_over, y_over, test, 7)
#xgb_train, xgb_test = get_stacking_base_datasets(xgb_wrapper, x_train, y_train, test, 7)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_wrapper, x_over, y_over, test,  7)    

CatBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
0:	learn: 0.6898463	total: 108ms	remaining: 43.2s
1:	learn: 0.6866037	total: 156ms	remaining: 31s
2:	learn: 0.6834290	total: 200ms	remaining: 26.4s
3:	learn: 0.6803059	total: 248ms	remaining: 24.5s
4:	learn: 0.6772377	total: 294ms	remaining: 23.2s
5:	learn: 0.6742306	total: 347ms	remaining: 22.8s
6:	learn: 0.6712770	total: 392ms	remaining: 22s
7:	learn: 0.6683845	total: 437ms	remaining: 21.4s
8:	learn: 0.6655401	total: 485ms	remaining: 21.1s
9:	learn: 0.6627518	total: 532ms	remaining: 20.7s
10:	learn: 0.6600157	total: 579ms	remaining: 20.5s
11:	learn: 0.6573285	total: 626ms	remaining: 20.2s
12:	learn: 0.6546942	total: 671ms	remaining: 20s
13:	learn: 0.6521072	total: 719ms	remaining: 19.8s
14:	learn: 0.6495730	total: 765ms	remaining: 19.6s
15:	learn: 0.6470860	total: 814ms	remaining: 19.5s
16:	learn: 0.6448202	total: 860ms	remaining: 19.4s
17:	learn: 0.6425936	total: 906ms	remaining: 19.2s
18:	learn: 0.6401994	total: 954ms	remaining: 19.1s


160:	learn: 0.4880109	total: 7.85s	remaining: 11.7s
161:	learn: 0.4876391	total: 7.9s	remaining: 11.6s
162:	learn: 0.4872348	total: 7.94s	remaining: 11.5s
163:	learn: 0.4868424	total: 7.99s	remaining: 11.5s
164:	learn: 0.4863342	total: 8.03s	remaining: 11.4s
165:	learn: 0.4852423	total: 8.09s	remaining: 11.4s
166:	learn: 0.4848768	total: 8.13s	remaining: 11.3s
167:	learn: 0.4844609	total: 8.18s	remaining: 11.3s
168:	learn: 0.4841014	total: 8.23s	remaining: 11.2s
169:	learn: 0.4836970	total: 8.42s	remaining: 11.4s
170:	learn: 0.4833007	total: 8.64s	remaining: 11.6s
171:	learn: 0.4828074	total: 8.71s	remaining: 11.5s
172:	learn: 0.4817555	total: 8.76s	remaining: 11.5s
173:	learn: 0.4813864	total: 8.83s	remaining: 11.5s
174:	learn: 0.4810336	total: 8.9s	remaining: 11.4s
175:	learn: 0.4806509	total: 8.97s	remaining: 11.4s
176:	learn: 0.4801704	total: 9.04s	remaining: 11.4s
177:	learn: 0.4798254	total: 9.11s	remaining: 11.4s
178:	learn: 0.4794860	total: 9.19s	remaining: 11.3s
179:	learn: 0.

319:	learn: 0.4338686	total: 19.8s	remaining: 4.95s
320:	learn: 0.4336529	total: 19.9s	remaining: 4.89s
321:	learn: 0.4334986	total: 19.9s	remaining: 4.83s
322:	learn: 0.4332871	total: 20s	remaining: 4.77s
323:	learn: 0.4330927	total: 20.1s	remaining: 4.71s
324:	learn: 0.4328954	total: 20.1s	remaining: 4.65s
325:	learn: 0.4327444	total: 20.2s	remaining: 4.59s
326:	learn: 0.4325321	total: 20.3s	remaining: 4.53s
327:	learn: 0.4323404	total: 20.4s	remaining: 4.47s
328:	learn: 0.4320617	total: 20.4s	remaining: 4.41s
329:	learn: 0.4318685	total: 20.5s	remaining: 4.35s
330:	learn: 0.4317183	total: 20.6s	remaining: 4.29s
331:	learn: 0.4314487	total: 20.7s	remaining: 4.23s
332:	learn: 0.4311830	total: 20.7s	remaining: 4.17s
333:	learn: 0.4309958	total: 20.8s	remaining: 4.11s
334:	learn: 0.4308060	total: 20.9s	remaining: 4.05s
335:	learn: 0.4304961	total: 21s	remaining: 3.99s
336:	learn: 0.4303494	total: 21s	remaining: 3.93s
337:	learn: 0.4301429	total: 21.1s	remaining: 3.87s
338:	learn: 0.4299

83:	learn: 0.5633564	total: 4.09s	remaining: 15.4s
84:	learn: 0.5625206	total: 4.14s	remaining: 15.3s
85:	learn: 0.5616750	total: 4.19s	remaining: 15.3s
86:	learn: 0.5608445	total: 4.23s	remaining: 15.2s
87:	learn: 0.5599706	total: 4.28s	remaining: 15.2s
88:	learn: 0.5591696	total: 4.33s	remaining: 15.1s
89:	learn: 0.5583767	total: 4.37s	remaining: 15.1s
90:	learn: 0.5575954	total: 4.42s	remaining: 15s
91:	learn: 0.5568099	total: 4.47s	remaining: 15s
92:	learn: 0.5560389	total: 4.52s	remaining: 14.9s
93:	learn: 0.5552137	total: 4.56s	remaining: 14.9s
94:	learn: 0.5544667	total: 4.61s	remaining: 14.8s
95:	learn: 0.5536642	total: 4.66s	remaining: 14.8s
96:	learn: 0.5528770	total: 4.71s	remaining: 14.7s
97:	learn: 0.5521566	total: 4.75s	remaining: 14.7s
98:	learn: 0.5514421	total: 4.8s	remaining: 14.6s
99:	learn: 0.5506832	total: 4.85s	remaining: 14.5s
100:	learn: 0.5499725	total: 4.9s	remaining: 14.5s
101:	learn: 0.5492728	total: 4.94s	remaining: 14.4s
102:	learn: 0.5485876	total: 4.99s	

243:	learn: 0.4832829	total: 14.8s	remaining: 9.49s
244:	learn: 0.4829435	total: 14.9s	remaining: 9.44s
245:	learn: 0.4826681	total: 15s	remaining: 9.38s
246:	learn: 0.4824208	total: 15.1s	remaining: 9.33s
247:	learn: 0.4821818	total: 15.1s	remaining: 9.27s
248:	learn: 0.4819098	total: 15.2s	remaining: 9.22s
249:	learn: 0.4815785	total: 15.3s	remaining: 9.16s
250:	learn: 0.4813124	total: 15.3s	remaining: 9.1s
251:	learn: 0.4810680	total: 15.4s	remaining: 9.05s
252:	learn: 0.4807238	total: 15.5s	remaining: 8.99s
253:	learn: 0.4803996	total: 15.5s	remaining: 8.93s
254:	learn: 0.4800617	total: 15.6s	remaining: 8.88s
255:	learn: 0.4798208	total: 15.7s	remaining: 8.82s
256:	learn: 0.4795854	total: 15.8s	remaining: 8.77s
257:	learn: 0.4793542	total: 15.8s	remaining: 8.71s
258:	learn: 0.4791271	total: 15.9s	remaining: 8.66s
259:	learn: 0.4787954	total: 16s	remaining: 8.6s
260:	learn: 0.4784784	total: 16s	remaining: 8.55s
261:	learn: 0.4781067	total: 16.1s	remaining: 8.49s
262:	learn: 0.477876

...

90:	learn: 0.5550642	total: 6.56s	remaining: 22.3s
91:	learn: 0.5542460	total: 6.63s	remaining: 22.2s
92:	learn: 0.5534792	total: 6.71s	remaining: 22.1s
93:	learn: 0.5525987	total: 6.78s	remaining: 22.1s
94:	learn: 0.5518043	total: 6.85s	remaining: 22s
95:	learn: 0.5510541	total: 6.92s	remaining: 21.9s
96:	learn: 0.5503148	total: 6.99s	remaining: 21.8s
97:	learn: 0.5495364	total: 7.06s	remaining: 21.8s
98:	learn: 0.5486909	total: 7.13s	remaining: 21.7s
99:	learn: 0.5478607	total: 7.2s	remaining: 21.6s
100:	learn: 0.5471424	total: 7.27s	remaining: 21.5s
101:	learn: 0.5464161	total: 7.35s	remaining: 21.5s
102:	learn: 0.5456696	total: 7.42s	remaining: 21.4s
103:	learn: 0.5449758	total: 7.49s	remaining: 21.3s
104:	learn: 0.5442754	total: 7.56s	remaining: 21.2s
105:	learn: 0.5435494	total: 7.63s	remaining: 21.2s
106:	learn: 0.5427633	total: 7.71s	remaining: 21.1s
107:	learn: 0.5420776	total: 7.79s	remaining: 21.1s
108:	learn: 0.5414106	total: 7.86s	remaining: 21s
109:	learn: 0.5407412	total

250:	learn: 0.4783669	total: 18.6s	remaining: 11s
251:	learn: 0.4780908	total: 18.7s	remaining: 11s
252:	learn: 0.4777957	total: 18.7s	remaining: 10.9s
253:	learn: 0.4775241	total: 18.8s	remaining: 10.8s
254:	learn: 0.4772457	total: 18.9s	remaining: 10.7s
255:	learn: 0.4769447	total: 18.9s	remaining: 10.7s
256:	learn: 0.4766550	total: 19s	remaining: 10.6s
257:	learn: 0.4763881	total: 19.1s	remaining: 10.5s
258:	learn: 0.4761093	total: 19.2s	remaining: 10.4s
259:	learn: 0.4757879	total: 19.2s	remaining: 10.4s
260:	learn: 0.4755247	total: 19.3s	remaining: 10.3s
261:	learn: 0.4752573	total: 19.4s	remaining: 10.2s
262:	learn: 0.4749739	total: 19.5s	remaining: 10.1s
263:	learn: 0.4746849	total: 19.5s	remaining: 10.1s
264:	learn: 0.4744250	total: 19.6s	remaining: 9.98s
265:	learn: 0.4741704	total: 19.7s	remaining: 9.91s
266:	learn: 0.4738921	total: 19.7s	remaining: 9.84s
267:	learn: 0.4736220	total: 19.8s	remaining: 9.76s
268:	learn: 0.4733620	total: 19.9s	remaining: 9.68s
269:	learn: 0.4730

9:	learn: 0.6683135	total: 732ms	remaining: 28.6s
10:	learn: 0.6661903	total: 806ms	remaining: 28.5s
11:	learn: 0.6640154	total: 880ms	remaining: 28.5s
12:	learn: 0.6616864	total: 952ms	remaining: 28.3s
13:	learn: 0.6596512	total: 1.02s	remaining: 28.3s
14:	learn: 0.6575625	total: 1.1s	remaining: 28.2s
15:	learn: 0.6553236	total: 1.17s	remaining: 28.2s
16:	learn: 0.6532899	total: 1.25s	remaining: 28.1s
17:	learn: 0.6511129	total: 1.32s	remaining: 28s
18:	learn: 0.6491860	total: 1.39s	remaining: 27.9s
19:	learn: 0.6472974	total: 1.46s	remaining: 27.8s
20:	learn: 0.6452030	total: 1.53s	remaining: 27.7s
21:	learn: 0.6433725	total: 1.61s	remaining: 27.7s
22:	learn: 0.6414927	total: 1.68s	remaining: 27.6s
23:	learn: 0.6396454	total: 1.75s	remaining: 27.4s
24:	learn: 0.6376504	total: 1.82s	remaining: 27.4s
25:	learn: 0.6356882	total: 1.9s	remaining: 27.3s
26:	learn: 0.6339151	total: 1.96s	remaining: 27.1s
27:	learn: 0.6321707	total: 2.03s	remaining: 27s
28:	learn: 0.6304930	total: 2.1s	remai

172:	learn: 0.5062066	total: 12.5s	remaining: 16.4s
173:	learn: 0.5057913	total: 12.6s	remaining: 16.4s
174:	learn: 0.5053856	total: 12.7s	remaining: 16.3s
175:	learn: 0.5049464	total: 12.8s	remaining: 16.2s
176:	learn: 0.5044892	total: 12.8s	remaining: 16.2s
177:	learn: 0.5040417	total: 12.9s	remaining: 16.1s
178:	learn: 0.5036103	total: 13s	remaining: 16s
179:	learn: 0.5031882	total: 13.1s	remaining: 16s
180:	learn: 0.5027880	total: 13.1s	remaining: 15.9s
181:	learn: 0.5023145	total: 13.2s	remaining: 15.8s
182:	learn: 0.5019202	total: 13.3s	remaining: 15.7s
183:	learn: 0.5014561	total: 13.3s	remaining: 15.7s
184:	learn: 0.5010516	total: 13.4s	remaining: 15.6s
185:	learn: 0.5006629	total: 13.5s	remaining: 15.5s
186:	learn: 0.5002492	total: 13.6s	remaining: 15.4s
187:	learn: 0.4998423	total: 13.6s	remaining: 15.4s
188:	learn: 0.4994510	total: 13.7s	remaining: 15.3s
189:	learn: 0.4990692	total: 13.8s	remaining: 15.2s
190:	learn: 0.4986497	total: 13.9s	remaining: 15.2s
191:	learn: 0.4982

334:	learn: 0.4580345	total: 24.2s	remaining: 4.7s
335:	learn: 0.4578355	total: 24.3s	remaining: 4.63s
336:	learn: 0.4575969	total: 24.3s	remaining: 4.55s
337:	learn: 0.4573981	total: 24.4s	remaining: 4.47s
338:	learn: 0.4572079	total: 24.4s	remaining: 4.4s
339:	learn: 0.4570401	total: 24.5s	remaining: 4.32s
340:	learn: 0.4568452	total: 24.5s	remaining: 4.25s
341:	learn: 0.4566410	total: 24.6s	remaining: 4.17s
342:	learn: 0.4564493	total: 24.6s	remaining: 4.09s
343:	learn: 0.4562508	total: 24.7s	remaining: 4.02s
344:	learn: 0.4560648	total: 24.7s	remaining: 3.94s
345:	learn: 0.4558685	total: 24.8s	remaining: 3.87s
346:	learn: 0.4557047	total: 24.8s	remaining: 3.79s
347:	learn: 0.4555115	total: 24.9s	remaining: 3.72s
348:	learn: 0.4553219	total: 24.9s	remaining: 3.64s
349:	learn: 0.4551343	total: 25s	remaining: 3.57s
350:	learn: 0.4549348	total: 25s	remaining: 3.5s
351:	learn: 0.4547265	total: 25.1s	remaining: 3.42s
352:	learn: 0.4545124	total: 25.2s	remaining: 3.35s
353:	learn: 0.45433

In [42]:

Stack_final_X_train = np.concatenate(( cat_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate(( cat_test, lgbm_test), axis=1)
print('원본 학습 피처 데이터 Shape:',x_train.shape, '원본 테스트 피처 Shape:',test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (911803, 74) 원본 테스트 피처 Shape: (506691, 50)
스태킹 학습 피처 데이터 Shape: (1139754, 2) 스태킹 테스트 피처 데이터 Shape: (506691, 2)


In [43]:
from sklearn.linear_model import LogisticRegression
lr_final = LogisticRegression()
lr_final.fit(Stack_final_X_train, y_over)
stack_final = lr_final.predict_proba(Stack_final_X_test)[:,1]

In [44]:
stack_final

array([0.19269801, 0.20331846, 0.14115454, ..., 0.15688354, 0.17410174,
       0.5885525 ])

In [46]:
submission = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/sample_submission.csv')

In [47]:
submission['isFraud']=stack_final

In [48]:
submission.to_csv('year_dream4.csv',index=False)